# System basics and stimulating organoids

The NeuroPlatform uses a dedicated API to control the equipment and to acquire data.

Below is an explanation of the system's main components and how to use them.




## Basic concepts

Let's start with a brief description of the components, and an introduction to the core concepts of the system.

Don't worry if not everything is clear, we will put everything in practice below where we show how to use the Python API to control the above system.

```{figure} ../_static/images/neuroplatform.png
---
height: 400px
name: neuroplatform-schema
---
Overview of the NeuroPlatform system.
```

- **Spike DB** : The database recording all data from the electrodes. This is active 24/7 and will record **spike events** from the electrodes.
  - A **spike event** is logged whenever the voltage from an electrode crosses a threshold of six times the standard deviation of the noise.
  - For an in-depth tutorial on using the database, see the [Database tutorial](db_usage).
- **Intan Software** : This software is connected to the MEAs and is used to configure the system and to acquire data. You will be interacting with this software via Python to control the system.
- **Intan Controller** : The hardware in charge of recording and stimulating the electrodes. This is the physical device that is connected to the MEAs.
- **Trigger Generator** : This device is used to generate triggers that are sent to the **Intan Controller**.
  - When you set a **StimParam** in Python, it will be sent to the Intan software. 
  - When you send a trigger, the trigger generator will send a signal to the **Intan Controller** to execute the parameter you set for that given trigger.

This brings us to the core concepts of the system:

- **StimParam** : This is a set of parameters that define a stimulation. It includes the duration, the amplitude, the frequency, and the electrode to stimulate.
- The StimParam is associated with a trigger. Whenever said trigger is sent, the **Intan Controller** will execute the **StimParam** associated with that trigger, on the electrode to which the **StimParam** is associated.
  - **Index** : The index of the electrode to stimulate. This is the number of the electrode in the MEA.
  - **Trigger** : The trigger is used by the **Trigger Generator** to send a signal to the **Intan Controller** to execute the **StimParam**. There is a maximum of 16 triggers that can be used, from 0 to 15.
    - For example, if you send trigger 0 with two **StimParam**s with `trigger_key = 0` on electrodes (`index`) 1 and 2 respectively, the **Intan Controller** will stimulate electrodes 1 and 2 using the specified parameters when it receives the trigger 0.

```{caution}
**DO NOT** use the same index in two different **StimParam**s. This will overwrite the previous **StimParam** and only use the last one.
```


Now, let's look at code to help you understand the sequence of events, as well as the limitations of the system.

### Creating experiments

To interact with the system, we use a system of tokens and "Experiments".

Simply use the provided token to create your experiment.
This is used to check that no other experiment is running while you use the system, and avoid conflicts.

```{note}
If you are unable to start your experiment during your booking because another experiment is running, please contact us.
```

#### Imports

In [ ]:
import numpy as np
import time
from datetime import datetime, timedelta
from IPython.display import clear_output
from neuroplatform import StimShape, StimParam , IntanSofware, Trigger, Database, StimPolarity, Experiment

#### Experiment and token

In [ ]:
token = "9T5KLS6T7X" # We provide you a token for the experiment
exp = Experiment(token)
print(f'Electrodes: {exp.electrodes}') # Electrodes that you can use

Using your token, you can show which electrodes you may use.

To see the live view of the electrode, go to the [Live view](https://livemea.finalspark.com/live) page.
Remember to check the "Absolute Index" function to get the correct electrode number.


```{warning}
When you start your experiment, always remember to stop it when you are done.
```

To start and stop your experiment, we recommend using `try... finally` blocks to ensure that the experiment is stopped even if an error occurs.

In [ ]:
try:
    exp.start()
    ... # Your experiment code here
finally:
    exp.stop() # This ensures a proper shutdown of the experiment

### Stimulation parameters

Now we can define the stimulation parameters.

Through Python, you will be controlling the Intan software to set the stimulation parameters.

The software provides a view of the parameters :

```{figure} ../_static/images/stimparam.png
---
height: 400px
name: stimparam
---
Stimulation parameters.
```

```{tip}
Feel free to use this as a visual reference when setting your parameters.
```

Stimulation parameters are defined by the following:

#### StimParam reference

| Name                     | Description                          | Default Value |
|--------------------------|--------------------------------------|---------------|
| `enable`                 | Enable stimulation                   | True          |
| `index`                  | Electrode for stimulation            | 64            |
| `trigger_key`            | Trigger key for stimulation          | 0             |
| `polarity`               | Polarity of the stimulation          | PositiveFirst |
| `phase_duration1`        | Duration of the first phase (us)     | 100.0         |
| `phase_amplitude1`       | Amplitude of the first phase (uA)    | 4.0           |
| `phase_duration2`        | Duration of the second phase (us)    | 100           |
| `phase_amplitude2`       | Amplitude of the second phase (uA)   | 2.0           |
|--------------------------|--------------------------------------|---------------|
| `stim_shape`             | Shape of the stimulation             | Biphasic      |
| `interphase_delay`       | Delay between phases (ms)            | 0.0           |
| `trigger_delay`          | Delay after trigger (ms)             | 0             |
| `nb_pulse`               | Number of pulses                     | 0             |
| `pulse_train_period`     | Period of pulse train (ms)           | 10000         |
|--------------------------|--------------------------------------|---------------|
| `post_stim_ref_period`   | Post-stimulation refractory period (ms) | 1000.0    |
| `enable_amp_settle`      | Enable amplitude settling            | True          |
| `pre_stim_amp_settle`    | Pre-stimulation amplitude settling (ms) | 0.0       |
| `post_stim_amp_settle`   | Post-stimulation amplitude settling (ms) | 1000.0    |
| `enable_charge_recovery` | Enable charge recovery               | True          |
| `post_charge_recovery_on`| Post-charge recovery on time (ms)    | 0.0           |
| `post_charge_recovery_off`| Post-charge recovery off time (ms)  | 100.0         |

```{seealso}
:class: dropdown
You can use the [StimParamLoader utility](stimparamloader) to help you check for inconsistencies.
```

We recommend the following :

- The first section of the table contains the basic parameters for the stimulation.
  - You likely will have to adapt these for each experiment.
  - If possible, balance the charge of the stimulation.
    - Ensure that $\text{phase_duration1} \times \text{phase_amplitude1} = \text{phase_duration2} \times \text{phase_amplitude2}$.
    - This improves the lifetime of both the organoid and the electrodes.
  - The polarity can greatly affect the response. Please remember to check which polarity is best for your experiment.
- The second section contains useful parameters you may want to modify for specific experiments.
  - The `nb_pulse` parameter is useful for burst stimulation.
  - The `pulse_train_period` parameter is useful for setting the period of the burst stimulation.
- The last section contains parameters that we recommend leaving as-is.
  

```{note}
Due to the network-based nature of the communication between components, using Python to time your stimulation will always be less precise than using the pulse train settings.
However, the pulse train settings may not be changed quickly during the experiment.
Therefore, you must choose between flexible spike trains with less precise timing or precise timing with less flexibility.
```

In [ ]:
stim_param1 = StimParam()
stim_param1.enable = True # Enable the stimulation
stim_param1.trigger_key = 0 # Trigger key to be used
stim_param1.index = 0 # Index of the stimulation
# --- Stimulation parameters --- #
stim_param1.polarity = StimPolarity.PositiveFirst
stim_param1.phase_duration2 = 100
stim_param1.phase_duration1 = 100
stim_param1.phase_amplitude2 = 100
stim_param1.phase_amplitude1 = 100
stim_param1.display_attributes()

In [ ]:
stim_param2 = StimParam()
stim_param2.index = exp.electrodes[8] # Use your provided electrodes to choose the site of stimulation

stim_param2.enable = True
stim_param2.trigger_key = 10 # The trigger key that will be used to send the stimulation
stim_param2.polarity = StimPolarity.NegativeFirst
stim_param2.display_attributes()

When you create a parameter, remember the concepts we previously showcased:

- **Index** : You specify the electrode you want to stimulate.
  - As mentioned, you cannot use two distinct parameters with the same index. This will overwrite the previous parameter.
- **Trigger key** : You specify the trigger that will be sent to the **Intan Controller**. The controller can accomodate **16** triggers.
- **Enable** : if the parameter is enabled, the **Intan Software** will record that a given parameter is set for a given trigger.

```{note}
When you finish your experiment, make sure to disable all **StimParam**s by setting `enable` to `False` and sending them to the **Intan Software**.
```

## Connecting to the Intan and the TriggerGenerator

Next, we will review the connection to the **IntanSoftware** and the **TriggerGenerator**, and how to send the parameters we created.

### Connecting to the Intan and sending parameters

To send the parameters to the **IntanSoftware**, simply do the following:


In [ ]:
intan = IntanSofware()
params = [stim_param1, stim_param2] # Create a list of stimulation parameters

intan.send_stimparam(params) # Send the stimulation parameters to the Intan software. THIS TAKES 10 SECONDS
...
intan.close() # Close the connection to the Intan software

```{caution}
Always close your connection to the Intan when you are done using `intan.close()`.
```

**Sending the parameters will take ten seconds to complete.** This is because the **IntanSoftware** will check the parameters and send them to the **Intan Controller**. 
Currently this cannot be bypassed, therefore parameters have to be set before your experiment starts, or during a ten-second pause in your experiment.

```{note}
Sending the parameters to the **IntanSoftware** will not stimulate the electrodes. You must send a trigger to the **TriggerGenerator** to stimulate the electrodes. See the next section for more information.
```

### Connecting to the TriggerGenerator and sending triggers
The main new concept here is how to send triggers.

When you send a trigger, the **TriggerGenerator** will send a signal to the **Intan Controller** to execute the **StimParam**s you set for that trigger.

We use a 16-length array of uint8 to send the triggers. If the value is 1, the trigger is sent. If the value is 0, the trigger is not sent.

For example, to send trigger 3, we would define the array as follows:

In [ ]:
trigger_gen 
trigger_array = np.zeros(16, dtype=np.uint8) # Here all the triggers are set to 0, so none will be sent

trigger_array[3] = 1 # This will send a trigger to the stimulation with trigger_key = 3
... # Run experiment...
trigger_gen.close() # Close the trigger generator

```{caution}
Always close your connection to the **TriggerGenerator** when you are done using `trigger_gen.close()`.
``` 

## Example experiment

Let's look at a small, simple example experiment, and put together all the concepts we have seen so far.

In [ ]:
intan = IntanSofware() # Here we connect to the Intan software
trigger_gen = Trigger() # Here we connect to the trigger generator

stim_params = [stim_param1, stim_param2]

try:
    if exp.start(): # Signal the start of an experiment to all users
        # Send stim parameter
        intan.send_stimparam(stim_params)
        start_exp = datetime.utcnow()
        print(start_exp)

        trigger0 = np.zeros(16, dtype=np.uint8)
        trigger0[0] = 1 # Enable stimulation on trigger 0, which is bound to stim_param1
        
        for i in range(10):
            # Stimulation on electrode 0
            trigger_gen.send(trigger0) # Here we send ten times the StimParam on electrode 0

            time.sleep(1)

        stop_exp = datetime.utcnow()
        print(stop_exp)

        # Disable all stims
        for stim in stim_params:
            stim.enable = False
        intan.send_stimparam(stim_params)

finally:
    # Close the connection to trigger generator
    trigger_gen.close()
    # Close the connection to intan software
    intan.close()
    # Signal the end of an experiment to all users
    exp.stop()

## Advanced functionalities

This section lists a few additional functionalities that you may find useful depending on your experiment.

### Tagging triggers

You can tag triggers with a number to identify them more easily.

When you retrieve the triggers in the database, you will see the tag you set. See the [Database tutorial](db_usage) for more information on how to do so.

In [ ]:
intan.set_tag_trigger(1) # Triggers will have this tag attached to them in the database

### Fixed threshold

You can set the threshold for spike events to be fixed, which means it will be constant throughout the experiment.

As mentioned earlier, the threshold is six times the standard deviation of the noise. This is recomputed when the variance threshold is enabled.

```{warning}
**ALWAYS** set back the variance threshold to **enabled** after finishing your experiment.
```

If you need to set a fixed threshold, for example to account for a drift in the noise or activity, you can use:

In [ ]:
try:
    ... # exp.start, other setup code
    intan.var_threshold(False) # Disable the variable threshold
    ... # your experiment code
finally:
    intan.var_threshold(True) # Enable the variable threshold
    ... # disable parameters, other cleanup code
    exp.stop() # Signal the end of the experiment

### Impedance measurement

You can ask the Intan to measure the impedance of the electrodes.
This takes **ten seconds** to complete, you can then retrieve the impedance values from the database. See the [Database tutorial](db_usage) for instructions.

In [ ]:
intan.impedance()